<a href="https://colab.research.google.com/github/iypc-team/NewNotebook/blob/master/New_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 11/2/2021-1
# https://www.tensorflow.org/tutorials/load_data/images
from __future__ import absolute_import
from IPython.display import clear_output
import glob, os, shutil, sys, uuid
from os.path import *
from time import sleep
# import TarfileFunctions
from TarfileFunctions import *
from BashColors import C
# from CV2_Utils import *
clear_output()
contentPath = os.getcwd()

os.chdir(contentPath)
cv2Path = join(contentPath, 'CV2Images')

if exists(cv2Path):
    shutil.rmtree(cv2Path)
    sleep(0.05)

if not exists(cv2Path):
    os.mkdir(cv2Path)
    tff.extractTarfiles('All_CV2_Images.tar.gz')

try:
    import tensorflow as tf
except Exception as err:
    print(err)
    %pip install tensorflow
    
import numpy as np

try: 
    from matplotlib import pyplot as plt
except Exception as err:
        %pip install matplotlib
        from matplotlib import pyplot as plt

os.chdir(contentPath)
clear_output()
import tensorflow as tf
print(f'tf version: {C.IBlue}{tf.__version__}{C.ColorOff}')
print(f'cwd: {C.IBlue}{os.getcwd()}{C.ColorOff}')

extracting: All_CV2_Images.tar.gz


FileNotFoundError: [Errno 2] No such file or directory: 'All_CV2_Images.tar.gz'

In [ ]:
os.chdir(cv2Path)
image_count = len(list(glob.glob('*/*.png', recursive=True)))
print(image_count, 'files')

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
if exists(cv2Path):
    print(cv2Path)
train_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgba',
    validation_split=0.2,
    subset="training",
    seed=456,
    image_size=(224, 224),
    batch_size=4)
print()

val_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgba',
    validation_split=0.2,
    subset="validation",
    seed=456,
    image_size=(224, 224),
    batch_size=4)

In [ ]:
class_names = train_ds.class_names
print(class_names)